# Import Needed Modules

In [124]:
# here we import some modules that we will need in the task 
import pandas as pd
import json
import numpy as np
from httpagentparser import simple_detect
from urllib.parse import urlparse
from datetime import datetime

# Load Json data

In [125]:
# make this function to choose the file we want to read from data directory 
def choose_json_file(id):
    while True:
        file_path = ""
        if id == 1:
            file_path = "Data/usa.gov_click_data_1.json"
            break
        elif id == 2:
            file_path = "Data/usa.gov_click_data_2.json"
            break
        elif id == 3:
            file_path = "Data/usa.gov_click_data_3.json"
            break
        else:
            print("Wrong Input Please Try Again")
            print()
            id = int(input("Please Choose File Number 1, 2 or 3 : "))
    return file_path

In [127]:
# Ask to Enter the ID that specify the direcotry of which file to read 
id = int(input("Please Choose File Number 1, 2 or 3 : "))
file_path = choose_json_file(id)
records = [json.loads(line) for line in open(file_path)]
# Print the first occurance
records[0]
print("The Length is : ", len(records))

Please Choose File Number 1, 2 or 3 :  3


The Length is :  3560


# Make Pandas Data Frame

In [128]:
# here we will make pandas data frame with columns names which we care about from json files keys 

# Create list of Columns Names
columns_names = ["Web Browser", "Operating System","From URL","To URL","City", "Longitude", "Latitude", "Time Zone", "Time In", "Time Out"]
json_df = pd.DataFrame(columns = columns_names)
json_df

,Web Browser,Operating System,From URL,To URL,City,Longitude,Latitude,Time Zone,Time In,Time Out


# Functions We Need in the Task

In [129]:
# here we will define some functions we will use in the task 

# use this function to get the operating system name and browser name of user_agent request string
def Get_Borwser_OperatingSystem(line, row_list):
    # check if the key of user_agent is not in the dictionary line 
    if 'a' not in line:
        # append null value of numpy in the list twice 
        row_list.append(np.nan)
        row_list.append(np.nan)
    else:
        # else we will append the detected browser and operating system using the http simple detector 
        row_list.append(simple_detect(line['a'])[0])
        row_list.append(simple_detect(line['a'])[1]) 
        
###########################################################################################################################################

# use This function to get the url to and from with regural expression of the records of json data 
def Get_To_From_URL(line, row_list):
    # First Focus on from URL key 
    # in case not found in the line dictionary 
    if 'r' not in line:
        # append nan in the list
        row_list.append(np.nan)
    # in case it is found
    else:
        # append the parsing of from url
        row_list.append(urlparse(line['r']).netloc)
        
    # Next Focus on To URL Key 
    # in case not found in the line dictionary 
    if 'u' not in line:
        # append nan in the list
        row_list.append(np.nan)
    else:
        # append the parsing of to url
        row_list.append(urlparse(line['u']).netloc)
###########################################################################################################################################

# use this function to get the city 
def Get_City(line, row_list):
    # in case there is no data of city 
    if 'cy' not in line:
        row_list.append(np.nan)
    # in case it is found
    else:
        # append the city name
        row_list.append(str(line['cy']))
        
###########################################################################################################################################

# use this function to get the longitude and latitude of the city 
def Get_Lat_Long(line, row_list):
    # in case the data is not found in json 
    if 'll' not in line:
        # append two nans to the list
        row_list.append(np.nan)
        row_list.append(np.nan)
    # in case the data is found
    else: 
        # append the longitude
        row_list.append(line['ll'][0])
        # append the latiude
        row_list.append(line['ll'][1])
###########################################################################################################################################

# use this function to get the data related to time zone, in and out 
def Get_Time_Data(line, row_list):
    # First work on the time zone
    # if the time zone not in the dictionary of json data
    if 'tz' not in line or str(line['tz']) == '':
        # append null
        row_list.append(np.nan)
    else:
        # append the time zone
        row_list.append(str(line['tz']))
        
    # Second work on start time
     # if the time of start not in the dictionary of json data
    if 't' not in line or str(line['t']) == '':
        # append null
        row_list.append(np.nan)
    else:
        # append the time after conversion from unix timestamp to original time only (extract the time from date)
        row_list.append(datetime.fromtimestamp(int(line['t'])).strftime("%H:%M:%S"))
    
    # Third work on Exit time
     # if the time of Exit not in the dictionary of json data
    if 'hc' not in line or str(line['hc']) == '':
        # append null
        row_list.append(np.nan)
    else:
        # append the time after conversion from unix timestamp to original time only (extract the time from date)
        row_list.append(datetime.fromtimestamp(int(line['hc'])).strftime("%H:%M:%S"))

# Fill Pandas Data Frame

In [130]:
# Loop on each line in jason file 

# Before This step make sure that pandas data frame is empty of rows 
json_df.drop(json_df.index, inplace=True)


for line in records: 
    # make list that will fill the row in pandas data frame
    row_list = [] 
    
    #########################################################################
    
    # Call Those Function to fill the list
    Get_Borwser_OperatingSystem(line, row_list)
    Get_To_From_URL(line, row_list)
    Get_City(line, row_list)
    Get_Lat_Long(line, row_list)
    Get_Time_Data(line, row_list)
    # print(len(row_list))
    #########################################################################
    
    # After we fill the list append it to the row in pandas data frame using the blank new row (length of pandas data frame) 
    json_df.loc[len(json_df)] = row_list

# After Finishing this show pandas data frame
json_df

,Web Browser,Operating System,From URL,To URL,City,Longitude,Latitude,Time Zone,Time In,Time Out
0,Windows 7,Chrome 17.0.963.78,www.facebook.com,www.ncbi.nlm.nih.gov,Danvers,42.576698,-70.954903,America/New_York,20:40:47,16:48:38
1,Unknown OS,Unknown Browser,www.AwareMap.com,www.monroecounty.gov,Provo,40.218102,-111.613297,America/Denver,20:40:49,00:13:13
2,Windows 7,Microsoft Internet Explorer 8.0,t.co,boxer.senate.gov,Washington,38.900700,-77.043098,America/New_York,20:40:50,19:45:41
3,MacOS Macintosh X 10.6.8,Safari 5.1.2,,apod.nasa.gov,Braz,-23.549999,-46.616699,America/Sao_Paulo,20:40:49,20:37:48
4,Windows 7,Chrome 17.0.963.79,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,20:40:51,15:53:31
...,...,...,...,...,...,...,...,...,...,...
3555,Windows 7,Microsoft Internet Explorer 9.0,,www.fda.gov,Paramus,40.944500,-74.070000,America/New_York,21:40:41,09:47:59
3556,Windows XP,Chrome 17.0.963.79,www.facebook.com,www.okc.gov,Oklahoma City,35.471500,-97.518997,America/Chicago,21:40:44,12:50:47
3557,Unknown OS,Unknown Browser,www.AwareMap.com,www.monroecounty.gov,Provo,40.218102,-111.613297,America/Denver,21:40:46,00:13:13
3558,Unknown OS,Unknown Browser,,www.ahrq.gov,Mountain View,37.419201,-122.057404,America/Los_Angeles,21:40:47,23:55:27


# Save Pandas Data Frame to CSV File

In [131]:
# First Specify the Path of the Save Directory  using the id we entered in the input section
Save_Path = 'target/Json_CSV_File_' + str(id)+".csv"
# Perform the Save To the Directory
json_df.to_csv(Save_Path, index=False)